In [2]:
# Import necessary libraries
import pandas as pd
import talib
from kiteconnect import KiteConnect
from apscheduler.schedulers.blocking import BlockingScheduler
import requests
import yfinance as yf
import json
import datetime
import telegram
from twilio.rest import Client
import openpyxl

# Zerodha API Credentials (Replace with your actual credentials)
api_key = "your_api_key"
access_token = "your_access_token"
kite = KiteConnect(api_key=api_key)
kite.set_access_token(access_token)

# Telegram and WhatsApp Configurations
TELEGRAM_BOT_TOKEN = "your_telegram_bot_token"
TELEGRAM_CHAT_ID = "your_telegram_chat_id"
WHATSAPP_SID = "your_twilio_sid"
WHATSAPP_AUTH_TOKEN = "your_twilio_auth_token"
TWILIO_WHATSAPP_NUMBER = "your_twilio_whatsapp_number"
YOUR_WHATSAPP_NUMBER = "your_whatsapp_number"

# Send Alerts Function
def send_alert(message):
    try:
        # Telegram Alert
        telegram_url = f"https://api.telegram.org/bot{TELEGRAM_BOT_TOKEN}/sendMessage"
        telegram_data = {"chat_id": TELEGRAM_CHAT_ID, "text": message}
        requests.post(telegram_url, data=telegram_data)
        
        # WhatsApp Alert
        client = Client(WHATSAPP_SID, WHATSAPP_AUTH_TOKEN)
        client.messages.create(
            from_=f"whatsapp:{TWILIO_WHATSAPP_NUMBER}",
            body=message,
            to=f"whatsapp:{YOUR_WHATSAPP_NUMBER}"
        )
    except Exception as e:
        print(f"Alert Error: {e}")

# Function to fetch NSE stocks dynamically
def get_nse_stocks():
    try:
        url = "https://www.nseindia.com/api/equity-stockIndices?index=NIFTY%2050"
        headers = {"User-Agent": "Mozilla/5.0"}
        response = requests.get(url, headers=headers)
        data = response.json()
        return [stock["symbol"] for stock in data["data"]]
    except Exception as e:
        print(f"Error fetching NSE stocks: {e}")
        return []

# Fetch stock data
def get_stock_data(stock):
    df = yf.download(f"{stock}.NS", period="6mo", interval="1d")
    return df

# Function to detect patterns
def detect_patterns(df):
    patterns = {
        "Bullish Engulfing": talib.CDLENGULFING(df['Open'], df['High'], df['Low'], df['Close']) > 0,
        "Morning Star": talib.CDLMORNINGSTAR(df['Open'], df['High'], df['Low'], df['Close']) > 0,
        "Three White Soldiers": talib.CDL3WHITESOLDIERS(df['Open'], df['High'], df['Low'], df['Close']) > 0,
        "Hammer": talib.CDLHAMMER(df['Open'], df['High'], df['Low'], df['Close']) > 0,
        "Piercing Line": talib.CDLPIERCING(df['Open'], df['High'], df['Low'], df['Close']) > 0,
        "Bearish Engulfing": talib.CDLENGULFING(df['Open'], df['High'], df['Low'], df['Close']) < 0,
        "Evening Star": talib.CDLEVENINGSTAR(df['Open'], df['High'], df['Low'], df['Close']) < 0,
        "Three Black Crows": talib.CDL3BLACKCROWS(df['Open'], df['High'], df['Low'], df['Close']) < 0,
        "Shooting Star": talib.CDLSHOOTINGSTAR(df['Open'], df['High'], df['Low'], df['Close']) < 0,
        "Dark Cloud Cover": talib.CDLDARKCLOUDCOVER(df['Open'], df['High'], df['Low'], df['Close']) < 0
    }
    for pattern, signal in patterns.items():
        if signal.iloc[-1]:
            return pattern
    return None

# Execute trade
def execute_trade(stock, pattern):
    if "Bullish" in pattern or "Morning" in pattern:
        order_type = "BUY"
    else:
        order_type = "SELL"
    
    message = f"{order_type} Signal: {stock} based on {pattern}"
    send_alert(message)
    print(message)

# Auto-Daily Trade Report
def generate_trade_report(trades):
    report_file = "daily_trade_report.xlsx"
    df = pd.DataFrame(trades, columns=["Stock", "Pattern", "Order Type", "Timestamp"])
    df.to_excel(report_file, index=False)
    
    # Send report via Telegram
    with open(report_file, "rb") as file:
        telegram_url = f"https://api.telegram.org/bot{TELEGRAM_BOT_TOKEN}/sendDocument"
        requests.post(telegram_url, files={"document": file}, data={"chat_id": TELEGRAM_CHAT_ID})

# Trading Bot Function
def trading_bot():
    stocks = get_nse_stocks()
    trades = []
    for stock in stocks:
        df = get_stock_data(stock)
        if df is not None and not df.empty:
            pattern = detect_patterns(df)
            if pattern:
                execute_trade(stock, pattern)
                trades.append([stock, pattern, "Executed", datetime.datetime.now()])
    generate_trade_report(trades)

# Scheduler to run every 5 minutes
scheduler = BlockingScheduler()
scheduler.add_job(trading_bot, "interval", minutes=5)
scheduler.start()

KeyboardInterrupt: 